Web Scrape for Bilibili, the key is the url, for bilibili it is hidden, luckily people on the internet provides ways to locate it, rest is just simple.

Next step is to analyse the data obtained. 

It would be also needed to have a data pipeline of sorts

### 弹幕： 需要 CID 号

reference: https://blog.csdn.net/gets_s/article/details/115279029?utm_medium=distribute.pc_relevant.none-task-blog-baidujs_title-1&spm=1001.2101.3001.4242

In [12]:
# -!- coding: utf-8 -!-
import requests
import re
import pandas as pd
import string
import jieba


def get_data(cid):
    # 分析网页，并获取网页文件
    url = 'https://comment.bilibili.com/{}.xml'.format(cid)
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"
    }
    response = requests.get(url,headers = headers).content.decode('utf-8')
    return response

def parse_html(response):
    # 解读网页文件，获取关键信息
    pattern = re.compile(r'<d p=".*?">(.*?)</d>')
    danmuku = re.findall(pattern,response)
    return danmuku

def save_data(danmuku,cid):
    # 保存数据
    Dict = {
        'danmuku' : danmuku
    }
    pd_data = pd.DataFrame(Dict)
    cid = str(cid)
    name = cid + 'danmuku.csv'
    path = '/Users/yuf/Desktop/Implementation/Hands Dirty/BIlibili-comment-file/{}'.format(name)
    pd_data.to_csv(path,index = False,header=False,mode='w',encoding='utf-8-sig')

def data_preprocess(danmuku,cid):
    cid = str(cid)
    name = cid + 'danmuku.csv'
    path = '/Users/yuf/Desktop/Implementation/Hands Dirty/BIlibili-comment-file/{}'.format(name)
    with open(path ,mode='r',encoding='utf-8') as f:
        # 加载用户自定义字典
        jieba.load_userdict (r'D:\PyCharm\Program\pachong\Day03\userdict')
        reader = f.read().replace('\n','')
        # 加载停用词词表
        stopwords = [line.strip() for line in open(r'D:\PyCharm\Program\pachong\Day03\stop_wordslst',encoding ='gbk').readlines()]
        # 去标点，去数字，去空白
        pun_num = string.punctuation + string.digits
        table = str.maketrans('','',pun_num)
        reader = reader.translate(table)
        seg_list = jieba.cut(reader,cut_all=False)
        sentence = ''
        for word in seg_list:
            if word not in stopwords and word.isspace() == False:
                sentence += word
                sentence += ','
        sentence = sentence[:-1]
        return sentence

def count_words(txt,cid):
    cid = str(cid)
    name = cid + '弹幕词汇数统计.csv'
    path = 'D:\PyCharm\Program\pachong\Day03\弹幕数据\{}'.format(name)
    aDict = {}
    words = txt.split(',')
    for word in words:
        aDict[word] = aDict.get(word,0) + 1
    pd_count = pd.DataFrame(aDict,index=['times']).T.sort_values('times',ascending=False)
    pd_count.to_csv(path)

if __name__ == "__main__":

    cid = int(input('请输入你想查询的视频CID号：'))
    response = get_data(cid)
    danmuku = parse_html(response)
    save_data(danmuku,cid)
    #sentence = data_preprocess(danmuku,cid)
    #count_words(sentence,cid)

请输入你想查询的视频CID号： 333938696


### 评论： 需要 OID 号

In [26]:
page=3
url1 = "https://api.bilibili.com/x/v2/reply?pn="+ str(page)+"&type=1&oid=545430857&sort=2"

In [14]:
import requests
import time
from bs4 import BeautifulSoup
import json

def get_html(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'user-agent':  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
    }

    r = requests.get(url, timeout=30,headers=headers)
    r.raise_for_status()
    r.endcodding = 'utf-8'
    return r.text


def get_content(url):
    comments = []
    html = get_html(url)
    try:
        s=json.loads(html)
    except:
        print("jsonload error")   
    num=len(s['data']['replies']) 
    i=0
    while i<num:
        comment=s['data']['replies'][i]
        InfoDict={}
        InfoDict['Uname']=comment['member']['uname']
        InfoDict['Like']=comment['like'] 
        InfoDict['Content']=comment['content']['message'] 
        InfoDict['Time']=time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(comment['ctime'])) 
        comments.append(InfoDict)
        i=i+1
    return comments


def Out2File(dict):
    with open('BiliBiliComments.txt', 'a+',encoding='utf-8') as f:
        i=0
        for comment in dict:
            i=i+1
            try:
                f.write('姓名：{}\t  点赞数：{}\t \n 评论内容：{}\t  评论时间：{}\t \n '.format(
                    comment['Uname'], comment['Like'], comment['Content'], comment['Time']))
                f.write("-----------------\n")
            except:
                print("out2File error")
        print('当前页面保存完成')

if __name__ == '__main__':
    e=0
    page=1
    while e == 0 :
        # seems to mimic the request URL for the network 'item' which contains reply
        # For instance it reads 'https://api.bilibili.com/x/v2/reply/subject/interaction-status?type=1&oid=545430857'
        url = "https://api.bilibili.com/x/v2/reply?pn="+ str(page)+"&type=1&oid=545430857&sort=2" 
        try:
            print()
            content=get_content(url)
            print("page:",page)
            Out2File(content)
            page=page+1
            # 为了降低被封ip的风险，每爬20页便歇5秒。
            if page%10 == 0:
                time.sleep(5)
        except:
            e=1


page: 1
当前页面保存完成

page: 2
当前页面保存完成

page: 3
当前页面保存完成



In [40]:
url = 'https://api.bilibili.com/x/v2/reply/subject/interaction-status?type=1&oid=545430857'
headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'user-agent':  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
    }

r = requests.get(url1, timeout=30, headers=headers)
r.raise_for_status()
r.encodding = 'utf-8'
html = r.text

s = json.loads(html)
s['data']['replies'][0]['content']['message']

'我最喜欢看御姐打架嫁了[滑稽]'